In [26]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [27]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [37]:
chars = sorted(list(set(''.join(words))))
str_to_inx = {str:inx for inx, str in enumerate(chars, start=1)}
str_to_inx['.'] = 0
inx_to_str = {inx:str for inx, str in str_to_inx.items()}
str_to_inx

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '.': 0}

In [40]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for word in words[:5]:
    word = block_size * '.' + word + '.'
    for ch1, ch2, ch3, ch4 in zip(word, word[1:], word[2:], word[3:]):
        # print(''.join([ch1, ch2, ch3]), '=>', ch4)
        X.append([str_to_inx[ch] for ch in [ch1, ch2, ch3]])
        Y.append(str_to_inx[ch4])

X, Y = torch.tensor(X), torch.tensor(Y)

In [42]:
X.dtype, X.shape, Y.dtype, Y.shape

(torch.int64, torch.Size([32, 3]), torch.int64, torch.Size([32]))